In [1]:
import mysql.connector
import csv
import pandas as pd
import math

In [2]:
myConnection = mysql.connector.connect( user = 'root',
password = "Henry999",
host = 'localhost',
database = 'Basketball')
cursorObject = myConnection.cursor()

In [3]:
# Read data from team.csv
with open('data/team.csv', mode='r') as team_file:
    team_reader = csv.DictReader(team_file)
    team_values = []
    for team_row in team_reader:
        team_id = team_row['id']
        full_name = team_row['full_name']
        abbreviation = team_row['abbreviation']
        city = team_row['city']
        year_founded = int(float(team_row['year_founded']))
        team_values.append((team_id, full_name, abbreviation, city, year_founded))
     
# Insert the data into the Team table
team_insert_query = "INSERT INTO Team VALUES (%s, %s, %s, %s, %s)"  
cursorObject.executemany(team_insert_query, team_values)
myConnection.commit()

In [4]:
teamsTEST = pd.read_csv('data/team.csv')
seasons = pd.read_csv('data/seasonwinners.csv').drop('Unnamed: 0', axis = 1)
seasons.loc[~seasons['Winner_ID'].isin(teamsTEST['id']), 'Winner_ID'] = 1610612744

cols = ['Season_ID', 'Winner_ID']
value_string = '%s, ' * len(cols)
value_string = value_string[:-2]
insert_games_query = f"INSERT INTO Season ({', '.join(cols)}) VALUES ({value_string})"
cursorObject.executemany(insert_games_query, seasons.loc[:,cols].values.tolist())
myConnection.commit()

In [5]:
# Read data from officials.csv
with open('data/officials_info.csv', mode='r') as officials_file:
    officials_reader = csv.DictReader(officials_file)
    officials_values = []
    for row in officials_reader:
        official_id = int(row['official_id'])
        referee_name = f"{row['first_name']} {row['last_name']}"
        officials_values.append((official_id, referee_name))

# Insert the data into the Referees table
insert_officials_query = "INSERT INTO Referees VALUES (%s, %s)"
cursorObject.executemany(insert_officials_query, officials_values)
myConnection.commit()

In [6]:
# games = pd.read_csv("data/game.csv")
# teamsTEST = pd.read_csv('data/team.csv')
# games = games.loc[games['team_id_home'].isin(teamsTEST['id'])]
# new_season_ids = []
# for season_id in games['season_id']:
#     year = season_id%100
#     if year >= 46 and year <= 99:
#         new_season_ids.append(1900+year)
#     else:
#         new_season_ids.append(2000+year)
# games['season_id'] = new_season_ids
# games = pd.read_csv("data/game.csv")
# games.loc[games['']]

In [7]:
games = pd.read_csv("data/game.csv").drop("Unnamed: 0", axis = 1)
teamsTEST = pd.read_csv('data/team.csv')
games = games.loc[games['team_id_away'].isin(teamsTEST['id'])]
games = games.loc[games['team_id_home'].isin(teamsTEST['id'])]
games.to_csv('data/game.csv')

In [8]:
seen = set()

# Read data from team_details.csv
with open('data/team_details.csv', mode='r') as team_details_file:
    team_details_reader = csv.DictReader(team_details_file)
    arenas_values = []
    arena_ids = {}
    abrev_to_arena_id = {}
    for row in team_details_reader:
        arena_id = hash(row['arena']) % (10 ** 4)
        arena_name = row['arena']
        arena_ids[arena_name] = arena_id
        abrev_to_arena_id[row['abbreviation']] = arena_id
        arena_capacity = int(float(row['arenacapacity'])) if row['arenacapacity'] else 0
        if arena_id not in seen:
            arenas_values.append((arena_id, arena_name, arena_capacity))
            seen.add(arena_id)

# Insert the data into the Arenas table
insert_arenas_query = "INSERT INTO Arenas VALUES (%s, %s, %s)"
cursorObject.executemany(insert_arenas_query, arenas_values)
myConnection.commit()

In [9]:
# Read data from game.csv

team_detailsTEST = pd.read_csv('data/team_details.csv')

with open('data/game.csv', mode='r') as game_file:
    game_reader = csv.DictReader(game_file)
    game_values = []
    for row in game_reader:
        game_id = int(row['game_id'])
        season_id = int(row['season_id'])
        game_date = row['game_date']
        season_type = row['season_type']
        home_points = int(float(row['pts_home']))
        away_points = int(float(row['pts_away']))
        home_team_id = int(row['team_id_home'])
        away_team_id = int(row['team_id_away'])
        home_win = home_points > away_points
        arena_name = team_detailsTEST.loc[team_detailsTEST['team_id'] == home_team_id, 'arena'].values
        arena_id = arena_ids[arena_name[0]]
        game_values.append((game_id, season_id, game_date, season_type, home_points, away_points, home_team_id, away_team_id, home_win, arena_id))

# Insert the data into the Games table
insert_games_query = "INSERT INTO Games VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
cursorObject.executemany(insert_games_query, game_values)
myConnection.commit()

In [10]:
# Read data from common_player_info.csv
with open('data/common_player_info.csv', mode='r') as player_info_file:
    player_info_reader = csv.DictReader(player_info_file)
    player_values = []
    for row in player_info_reader:
        player_id = int(row['person_id'])
        first_name = row['first_name']
        last_name = row['last_name']
        full_name = f"{first_name} {last_name}"
        birth_date = row['birthdate']
        school = row['school']
        country = row['country']
        draft_year = row['draft_year']
        draft_round = row['draft_round']
        if draft_year == 'Undrafted':
            draft_year = None
            draft_round = None
        if draft_round == '':
            draft_round = None
        player_values.append((player_id, full_name, birth_date, school, country, draft_year, draft_round))

# Insert the data into the Players table
insert_players_query = "INSERT INTO Players VALUES (%s, %s, %s, %s, %s, %s, %s)"
cursorObject.executemany(insert_players_query, player_values)
myConnection.commit()


In [11]:
import pandas as pd

# Read data from SeasonPlayerStats.csv
playerseason = pd.read_csv('data/SeasonPlayerStats.csv')

# Function to extract the first year from SEASON_ID
def extract_first_year(season_id):
    return season_id.split('-')[0]

# Apply the function to the SEASON_ID column
playerseason['SEASON_ID'] = playerseason['SEASON_ID'].apply(extract_first_year)

# Filter out the 2023 season and select specific columns
cols = ['PLAYER_ID', 'SEASON_ID', 'TEAM_ID', 'PLAYER_AGE', 'GP', 'PTS', 'REB', 'AST', 'STL', 'BLK']
playerseason = playerseason.loc[playerseason['SEASON_ID'] != "2023"]
playerseason = playerseason.loc[:, cols]

# Handle missing values by filling them with a default value or dropping them
# Example: Filling NA values with 0
playerseason = playerseason.fillna(0)

# Define the data types for each column
dtype_dict = {
    'PLAYER_ID': 'int64',
    'SEASON_ID': 'str',
    'TEAM_ID': 'int64',
    'GP': 'int64',
    'AST': 'int64',
    'REB': 'int64',
    'STL': 'int64',
    'PTS': 'int64',
    'PLAYER_AGE': 'int64',
    'BLK': 'int64',
}

# Convert data types
playerseason = playerseason.astype(dtype_dict)

# Rename columns for clarity
playerseason = playerseason.rename(columns={
    'PLAYER_AGE': 'AGE',
    'GP': 'Games_Played',
    'PTS': 'Points',
    'REB': 'Rebounds',
    'STL': 'Steals',
    'BLK': 'Blocks',
    'AST': 'Assists'
})

# Query the Players table to get all valid PLAYER_IDs
select_players_query = "SELECT Player_ID FROM Players"
cursorObject.execute(select_players_query)
valid_player_ids = set(row[0] for row in cursorObject.fetchall())

# Query the Teams table to get all valid Team_IDS
select_teams_query = "SELECT Team_ID FROM Team"
cursorObject.execute(select_teams_query)
valid_teams_ids = set(row[0] for row in cursorObject.fetchall())

# Filter playerseason to only include rows with valid PLAYER_IDs
playerseason = playerseason[playerseason['PLAYER_ID'].isin(valid_player_ids)]

# Filter playerseason to only include rows with valid Team_IDs
playerseason = playerseason[playerseason['TEAM_ID'].isin(valid_teams_ids)]

# Insert the data into the PlayedSeasonWith table
cols = ['PLAYER_ID', 'SEASON_ID', 'TEAM_ID', 'AGE', 'Games_Played', 'Points', 'Rebounds', 'Assists', 'Steals', 'Blocks']
value_string = '%s, ' * len(cols)
value_string = value_string[:-2]
insert_games_query = f"INSERT INTO PlayedSeasonWith ({', '.join(cols)}) VALUES ({value_string})"
cursorObject.executemany(insert_games_query, playerseason.values.tolist())
myConnection.commit()


In [12]:
with open('data/team_details.csv', mode='r') as team_details_file:
    team_details_reader = csv.DictReader(team_details_file)
    home_venue_values = []
    for row in team_details_reader:
        team_id = int(row['team_id'])
        arena_id = arena_ids[(row['arena'])]  # Assuming 'arena' column contains Arena_ID
        home_venue_values.append((team_id, arena_id))

insert_home_venue_query = "INSERT INTO HomeVenue VALUES (%s, %s)"
cursorObject.executemany(insert_home_venue_query, home_venue_values)
myConnection.commit()

In [13]:
officiates = pd.read_csv("data/officials.csv")
officialsTest = pd.read_csv("data/officials_info.csv")
gamesTest = pd.read_csv('data/game.csv')
officiates = officiates.loc[officiates['game_id'].isin(gamesTest['game_id'])]
officiates = officiates.loc[officiates['official_id'].isin(officialsTest['official_id'])]

In [14]:
officiates_values = []
for idx, row in officiates.iterrows():
    game_id = int(row['game_id'])
    official_id = int(row['official_id'])
    jersey_number = int(row['jersey_num']) if  not math.isnan(row['jersey_num']) else None
    officiates_values.append((official_id, game_id, jersey_number))

insert_officiates_query = "INSERT INTO Officiates VALUES (%s, %s, %s)"
cursorObject.executemany(insert_officiates_query, officiates_values)
myConnection.commit()

In [15]:
cursorObject.execute(f"""
        SELECT 
            t.Team_Name,
            s.Season_ID,
            COUNT(g.Game_ID) AS Games_Played,
            SUM(CASE WHEN g.Home_Team_ID = s.Winner_ID THEN g.Home_Points ELSE g.Away_Points END) AS Points_Scored,
            SUM(CASE WHEN g.Home_Team_ID = s.Winner_ID THEN g.Away_Points ELSE g.Home_Points END) AS Points_Allowed,
            SUM(CASE WHEN g.Home_Team_ID = s.Winner_ID AND g.Home_Win = 1 THEN 1
                     WHEN g.Away_Team_ID = s.Winner_ID AND g.Home_Win = 0 THEN 1 ELSE 0 END) AS Wins,
            SUM(CASE WHEN g.Home_Team_ID = s.Winner_ID AND g.Home_Win = 0 THEN 1
                     WHEN g.Away_Team_ID = s.Winner_ID AND g.Home_Win = 1 THEN 1 ELSE 0 END) AS Losses
        FROM 
            Season s
        JOIN 
            Team t ON s.Winner_ID = t.Team_ID
        JOIN 
            Games g ON (g.Home_Team_ID = s.Winner_ID OR g.Away_Team_ID = s.Winner_ID) AND YEAR(g.Game_Date) = s.Season_ID
        WHERE 
            s.Season_ID BETWEEN 2015 AND 2023
        GROUP BY 
            t.Team_Name, s.Season_ID
        ORDER BY s.Season_ID;
    """)

In [16]:
cursorObject.fetchall()

[('Cleveland Cavaliers',
  2015,
  107,
  Decimal('10840'),
  Decimal('10361'),
  Decimal('71'),
  Decimal('36')),
 ('Golden State Warriors',
  2016,
  115,
  Decimal('13123'),
  Decimal('12011'),
  Decimal('93'),
  Decimal('22')),
 ('Golden State Warriors',
  2017,
  106,
  Decimal('12270'),
  Decimal('11133'),
  Decimal('85'),
  Decimal('21')),
 ('Toronto Raptors',
  2018,
  96,
  Decimal('10756'),
  Decimal('10215'),
  Decimal('66'),
  Decimal('30')),
 ('Los Angeles Lakers',
  2019,
  78,
  Decimal('8681'),
  Decimal('8657'),
  Decimal('42'),
  Decimal('36')),
 ('Milwaukee Bucks',
  2020,
  56,
  Decimal('6528'),
  Decimal('6230'),
  Decimal('33'),
  Decimal('23')),
 ('Golden State Warriors',
  2021,
  107,
  Decimal('12104'),
  Decimal('11577'),
  Decimal('69'),
  Decimal('38')),
 ('Denver Nuggets',
  2022,
  94,
  Decimal('10943'),
  Decimal('10724'),
  Decimal('58'),
  Decimal('36'))]